In [91]:
import numpy as np 
import pandas as pd
import os
import time
import ast

### Treat all the variables for csv and excel file


In [92]:

time0= time.time()

def get_last_added_file(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    
    # Filter Excel files
    excel_files = [file for file in files if file.endswith('.xlsx') or file.endswith('.xls') or file.endswith('.csv')]
    
    if not excel_files:
        return None  # No Excel files found
    print()
    # Sort files by modification time (newest first)
    excel_files.sort(key=lambda x: os.path.getctime(os.path.join(folder_path, x)), reverse=True)
    
    # Return the path to the last added Excel file
    return os.path.join(folder_path, excel_files[0])





def create_folder_if_not_exists(folder_name):
    # Get current folder 
    folder_path = os.getcwd()
    
    try:
        os.makedirs("pip")
        print("DONE")
    except FileExistsError:
        
        pass
    if not os.path.exists(os.path.join(folder_path,folder_name)):
        os.makedirs(folder_name)
        print("Folder '{}' created.".format(folder_name))
    else:
        print("Folder '{}' already exists.".format(folder_name))

# Folder name to create 
folder_name_to_convert = 'File_To_Convert'
folder_name_converted = 'File_Converted'

# Get current folder 
folder_path = os.getcwd()


# Create new folder 
create_folder_if_not_exists(folder_name_converted)



Folder 'File_Converted' already exists.


In [93]:
# Get the last upload excel file
real_folder_path= os.path.join(folder_path , folder_name_to_convert)

last_file = get_last_added_file(real_folder_path)


if last_file:
    print("Last added file:", last_file)
else:
    print("No files found in the folder.")

# Get the correct naming for the converted file 
filename_with_extension = os.path.basename(last_file)
splited_filename_with_extension = os.path.splitext(filename_with_extension)
print(splited_filename_with_extension)
filename = splited_filename_with_extension[0] + 'converted' + splited_filename_with_extension[1]

# Possibility to create a thread to run all the code during the wait for the user answer. 
# scenario = input("Give the name of the scenario")
scenario = splited_filename_with_extension[0]

# Read the selected file 
if splited_filename_with_extension[1] == '.xls' or splited_filename_with_extension[1] == '.xlsx':
    scenario_variable_df = pd.read_excel(last_file)
elif  splited_filename_with_extension[1] == '.csv': 
    scenario_variable_df = pd.read_csv(last_file)


scenario_variable_df


Last added file: c:\Users\AdGeeds\Documents\IAMC-format\IAMC_format\Conversion-Script\File_To_Convert\Baseline no policies no CO2tax.csv
('Baseline no policies no CO2tax', '.csv')


C:\Users\AdGeeds\AppData\Local\Temp\ipykernel_6040\2027774379.py:26: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  scenario_variable_df = pd.read_csv(last_file)


,Time,Year,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,2005,2006,2007,2008,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
0,world_PE_by_commodity,EJ/Year,PE_agriculture_products,NaN,NaN,NaN,0.028242,1.057450,1.265620,1.713050,...,2.948760,2.824390,2.727550,2.623520,2.632230,2.47606,2.401450,2.344890,2.263520,2.172020
1,world_PE_by_commodity,EJ/Year,PE_coal,NaN,NaN,NaN,139.888000,142.536000,145.529000,148.652000,...,239.392000,238.914000,242.655000,247.283000,252.159000,248.07700,251.761000,254.646000,253.793000,249.484000
2,world_PE_by_commodity,EJ/Year,PE_oil,NaN,NaN,NaN,202.483000,200.246000,200.156000,199.572000,...,170.113000,162.348000,159.496000,158.317000,158.668000,151.41400,150.100000,147.932000,144.727000,139.896000
3,world_PE_by_commodity,EJ/Year,PE_forestry_products,NaN,NaN,NaN,37.127500,37.334300,37.479100,37.766300,...,98.841200,98.722100,101.700000,105.143000,108.929000,106.94800,109.672000,112.871000,112.351000,111.132000
4,world_PE_by_commodity,EJ/Year,PE_geothermal,NaN,NaN,NaN,0.240555,0.249827,0.257823,0.267857,...,7.077130,6.729580,6.799890,6.949150,7.092140,6.82465,6.960060,6.905350,6.840980,6.567220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171137,Fe_hidden_resources,Mt,LOW_GRADE,NaN,NaN,NaN,128992.000000,127935.000000,126830.000000,125688.000000,...,66756.000000,65861.200000,64974.700000,64096.800000,63227.400000,62366.50000,61514.200000,60670.500000,59835.300000,59008.800000
171138,Fe_hidden_resources,Mt,ULTRALOW_GRADE,NaN,NaN,NaN,176000.000000,175990.000000,175978.000000,175963.000000,...,132301.000000,130580.000000,128867.000000,127163.000000,125469.000000,123787.00000,122116.000000,120458.000000,118813.000000,117183.000000
171139,Fe_hidden_resources,Mt,TRACE_GRADE,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
171140,Fe_hidden_resources,Mt,OCEANS_GRADE,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [94]:

#Rename the column of the file 
columns = scenario_variable_df.columns.str.replace('Unnamed:', 'Subscript')
columns = columns.str.replace('.', ' ')

# Define the string you want in column names
search_string = 'Subscript'
counter=0 

for k in range(len(columns)): 
    if search_string in columns[k]: 
        columns.values[k]= search_string + ' ' + str(counter)
        counter+=1



# Create a dictionary using zip() and dictionary comprehension
my_dict = {k: v for k, v in zip(scenario_variable_df.columns[2:6], columns[2:6])}
scenario_variable_df.rename(columns=my_dict, inplace=True)
scenario_variable_df.rename(columns={"Time": "Variable", "Year": "Unit", "Subscript":"Subscript 0"}, inplace=True)

# Insert the three columns in the right place for IAMC format
scenario_variable_df.insert(0, 'Model', "WILIAM")
scenario_variable_df.insert(1, 'Scenario', scenario)
scenario_variable_df.insert(2,"Region","World")


### Read the Excel File to change the name of the variable


In [95]:
# Read the CSV file into a pandas DataFrame
data_name_df = pd.read_excel('Variable_Reference/Variable_name_IAMC.xlsx')
# Remplacer les tirets du bas par des espaces dans la colonne
# data_name_df['WILIAM_variable'] = data_name_df['WILIAM_variable'].str.replace('_', ' ')
# Drop rows with NaN values which corresponds to values not conserv for the final upload of data in IAMC format
data_name_df.dropna(subset=['IAMC_variable'],inplace=True)
data_name_df

,IAMC_variable,IAMC_unit,WILIAM_variable,WILIAM_unit
0,Primary Energy,EJ/yr,world_PE_by_commodity,NaN
1,Primary Energy,EJ/yr,total_PE_by_region,NaN
2,Price|Primary Energy,NaN,fuel_price_by_PROTRA_world,Mdollars/EJ
3,Price|Primary Energy,NaN,fuel_price_by_PROTRA_9R_adjusted,Mdollars/EJ
4,Index Price|Final Energy,NaN,price_final_energy,Dmnl
...,...,...,...,...
161,Resource|Iron|Reserve Production Ratio,NaN,Fe_reserves_to_production_ratio,Year
162,Resource|Extraction|Iron,Mt,Fe_total_mining,Mt/Years
163,Resource|Market|Sales|Iron,NaN,Fe_market_sales,Mt/Year
164,Resource|Iron,Mt,Fe_hidden_resources,Mt


In [96]:
# Create a dict with Wiliam's name as key, IAMC's name as value
IAMC_WILIAM_name_dict = data_name_df.set_index('WILIAM_variable')['IAMC_variable'].to_dict()
print(IAMC_WILIAM_name_dict)
# Replace the variable name used in William to the ones used for IAMC format. 
scenario_variable_df['Variable']= scenario_variable_df['Variable'].replace(IAMC_WILIAM_name_dict)

# Keep only the rows where the value in column 'Variable' belongs to the list of wanted values 
variable_required_list = data_name_df['IAMC_variable'].to_list()
scenario_variable_df = scenario_variable_df[scenario_variable_df['Variable'].isin(variable_required_list)]

{'world_PE_by_commodity': 'Primary Energy', 'total_PE_by_region': 'Primary Energy', 'fuel_price_by_PROTRA_world': 'Price|Primary Energy', 'fuel_price_by_PROTRA_9R_adjusted': 'Price|Primary Energy', 'price_final_energy': 'Index Price|Final Energy', 'transport_electrified_vehicle_batteries_power': 'Capacity|Electricity|Power|Battery Electric Vehicles', 'CO2_tax_rate_sectors': 'Price|Carbon|Industries', 'GHG_emissions_1R': 'Emissions', 'GHG_emissions_1R_CO2_equivalent': 'Emissions|CO2eq', 'total_GHG_emissions': 'Emissions', 'total_CO2e_emissions_per_capita_9R': 'Emissions|CO2eq|Per capita', 'GHG_emissions_per_capita_9R': 'Emissions|Per capita', 'temperature_change': 'Temperature|Global Mean Increase', 'sea_level_rise_by_region': 'Sea level|Region|Mean rise', 'sea_level_rise': 'Sea level|Global Mean rise', 'pH_ocean': 'pH|Ocean', 'atmospheric_concentrations_CO2': 'Concentration|CO2', 'CO2_land_use_change_emissions': 'Emissions|CO2|Land use changes', 'CO2_IPPUs_emissions_9R': 'Emissions|CO2

### Check of the variable included in the file


In [97]:
variables_used_list = scenario_variable_df['Variable'].to_list()
variables_required_not_in = list(set(variable_required_list) - set(variables_used_list))
print('The missing variable in the export dataset are the following:', variables_required_not_in)
print('The number of missing varibles is ', len(variables_required_not_in))

The missing variable in the export dataset are the following: ['Output', 'Value Added|Taxes on products', 'Value Added|Net operating surplus', 'Households|Savings', 'Value Added|Consumption Fixed Capital', 'Capital Stock', 'Final Demand|Other', 'Households|Consumption|COICOP', 'Revenue|Households|Disposable per capita', 'Value Added|Labour Compensation', 'Revenues|Households|Disposable income', 'Value Added|Industry', 'Capital Formation', 'Value Added|Extraction', 'GDP|PPP', 'Value Added|Industry|Energy', 'Value Added|Taxes on production', 'Expenditure|Household', 'Households|Consumption nominal|COICOP', 'Value Added|Services', 'Public Consumption', 'GDP|Country|Per capita', 'Value Added|Agriculture', 'Capacity|Batteries|Transport Vehicule', 'Households|Savings|Real', 'Government|Budget Balance to GDP']
The number of missing varibles is  26


### Change the naming of Subcript in William


In [98]:
# Open the text file containing the energy dictionary
with open('Create_Variable_Dict/energy_dict.txt', 'r') as f:
    # Read the contents of the file
    energy_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
energy_dict = ast.literal_eval(energy_dict_str)


# Open the text file containing the rest dictionary
with open('Create_Variable_Dict/rest_dict.txt', 'r') as f:
    # Read the contents of the file
    rest_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
rest_dict = ast.literal_eval(rest_dict_str)



### Change Unit


In [99]:
def change_unit(row):
    if not pd.isna(row):
        if 'Year' in row:
            row = row.replace('Year','yr')
    return row 
scenario_variable_df['Unit'] = scenario_variable_df['Unit'].apply(change_unit)

C:\Users\AdGeeds\AppData\Local\Temp\ipykernel_6040\146703155.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scenario_variable_df['Unit'] = scenario_variable_df['Unit'].apply(change_unit)


### Order Dataframe to the required Format by adding subscript at the end of the variable


In [100]:
# Dict used to check if the subscripts are a country
country_dict= {'EU27':1, 'UK':1,'CHINA':1,'EASOC':1,'INDIA':1,'LATAM':1,'RUSSIA':1,'USMCA':1,'LROW':1,'AUSTRIA':1,'BELGIUM':1,'BULGARIA':1,'CROATIA':1,'CYPRUS':1,'CZECH_REPUBLIC':1,'DENMARK':1,'ESTONIA':1,'FINLAND':1,'FRANCE':1,'GERMANY':1,'GREECE':1,'HUNGARY':1,'IRELAND':1,'ITALY':1,'LATVIA':1,'LITHUANIA':1,'LUXEMBOURG':1,'MALTA':1,'NETHERLANDS':1,'POLAND':1,'PORTUGAL':1,'ROMANIA':1,'SLOVAKIA':1,'SLOVENIA':1,'SPAIN':1,'SWEDEN':1}

# Aggregate the subscripts at the end of the variable 
def aggregate_variable_name(row, counter):
    for k in range(counter):
        subscript=row["Subscript "+str(k)]
    
        if k==0 :
            if country_dict.get(subscript)==1: 
                #Give the name of the region for that variable
                row["Region"]=subscript
                
                row["Subscript 0"]=np.nan
                continue
        
        if not pd.isnull(subscript):
            
            if energy_dict.get(subscript, None) is not None:
                
                subscript = energy_dict[subscript]
            elif rest_dict.get(subscript,None): 
                subscript = rest_dict[subscript]
            
            row["Variable"]=row["Variable"]+"|"+subscript
            
    return row 


# Apply the previous function on all the lines 
scenario_variable_df= scenario_variable_df.apply(aggregate_variable_name, args=(counter,),axis=1)

# Remove the subscript columns once they have been added at the end of the variable name
drop_columns_list=["Subscript " + str(k) for k in range(counter)]
scenario_variable_df.drop(columns=drop_columns_list, inplace=True)

scenario_variable_df

,Model,Scenario,Region,Variable,Unit,2005,2006,2007,2008,2009,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
0,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Agriculture Products,EJ/yr,0.028242,1.057450,1.265620,1.713050,1.934410,...,2.948760,2.824390,2.727550,2.623520,2.632230,2.47606,2.401450,2.344890,2.263520,2.172020
1,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Coal,EJ/yr,139.888000,142.536000,145.529000,148.652000,149.764000,...,239.392000,238.914000,242.655000,247.283000,252.159000,248.07700,251.761000,254.646000,253.793000,249.484000
2,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Oil,EJ/yr,202.483000,200.246000,200.156000,199.572000,199.148000,...,170.113000,162.348000,159.496000,158.317000,158.668000,151.41400,150.100000,147.932000,144.727000,139.896000
3,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Forestry Products,EJ/yr,37.127500,37.334300,37.479100,37.766300,37.924300,...,98.841200,98.722100,101.700000,105.143000,108.929000,106.94800,109.672000,112.871000,112.351000,111.132000
4,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Geothermal,EJ/yr,0.240555,0.249827,0.257823,0.267857,0.278548,...,7.077130,6.729580,6.799890,6.949150,7.092140,6.82465,6.960060,6.905350,6.840980,6.567220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171137,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|LOW_GRADE,Mt,128992.000000,127935.000000,126830.000000,125688.000000,124519.000000,...,66756.000000,65861.200000,64974.700000,64096.800000,63227.400000,62366.50000,61514.200000,60670.500000,59835.300000,59008.800000
171138,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|ULTRALOW_GRADE,Mt,176000.000000,175990.000000,175978.000000,175963.000000,175945.000000,...,132301.000000,130580.000000,128867.000000,127163.000000,125469.000000,123787.00000,122116.000000,120458.000000,118813.000000,117183.000000
171139,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|TRACE_GRADE,Mt,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
171140,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|OCEANS_GRADE,Mt,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [101]:
# List with the words in uppercase which will not be capitalized by the next functions
upper_word_list = ['CO2', 'CH4', 'N2O', 'PFC', 'SF6', 'HFC134a', 'HFC23', 'HFC32', 'HFC125', 'HFC143a', 'HFC152a', 'HFC227ea', 'HFC245ca', 'HFC43-10', 'HFC', 'w/o CCS' ,'w/ CCS', 'PV', 'CSP', 'AFOLU', 'CO2eq', 'EROI']


# Change the format of the string to respect IAMC's format
def transform_string(s):
    # Split the string into words
    if pd.isnull(s): 
        return s
    words = s.split('|')

    capitalized_words = []
    # Capitalize the first letter of each word
    for word in words: 
        if word not in upper_word_list:
            new_word_list = word.split('_')
            new_word_list = [new_word.capitalize() for new_word in new_word_list] 
            capitalized_words += [' '.join(new_word_list) + '|']
        else: 
            capitalized_words += [word + '|']
    
    # Join the words with spaces
    transformed_string = ''.join(capitalized_words)

    result = ''
    capitalize_next=False
    # Capitalize the letter after each space
    for char in transformed_string[:-1]:
        if char == ' ' or char == '-':
            capitalize_next = True
            result += char
        elif capitalize_next:
            result += char.upper()
            capitalize_next = False
        else:
            result += char
                
    
    return result

# Capitalize each variable's name
scenario_variable_df['Variable'] = scenario_variable_df['Variable'].apply(transform_string)
scenario_variable_df

,Model,Scenario,Region,Variable,Unit,2005,2006,2007,2008,2009,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
0,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Agriculture Products,EJ/yr,0.028242,1.057450,1.265620,1.713050,1.934410,...,2.948760,2.824390,2.727550,2.623520,2.632230,2.47606,2.401450,2.344890,2.263520,2.172020
1,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Coal,EJ/yr,139.888000,142.536000,145.529000,148.652000,149.764000,...,239.392000,238.914000,242.655000,247.283000,252.159000,248.07700,251.761000,254.646000,253.793000,249.484000
2,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Oil,EJ/yr,202.483000,200.246000,200.156000,199.572000,199.148000,...,170.113000,162.348000,159.496000,158.317000,158.668000,151.41400,150.100000,147.932000,144.727000,139.896000
3,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Forestry Products,EJ/yr,37.127500,37.334300,37.479100,37.766300,37.924300,...,98.841200,98.722100,101.700000,105.143000,108.929000,106.94800,109.672000,112.871000,112.351000,111.132000
4,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Geothermal,EJ/yr,0.240555,0.249827,0.257823,0.267857,0.278548,...,7.077130,6.729580,6.799890,6.949150,7.092140,6.82465,6.960060,6.905350,6.840980,6.567220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171137,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Low Grade,Mt,128992.000000,127935.000000,126830.000000,125688.000000,124519.000000,...,66756.000000,65861.200000,64974.700000,64096.800000,63227.400000,62366.50000,61514.200000,60670.500000,59835.300000,59008.800000
171138,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Ultralow Grade,Mt,176000.000000,175990.000000,175978.000000,175963.000000,175945.000000,...,132301.000000,130580.000000,128867.000000,127163.000000,125469.000000,123787.00000,122116.000000,120458.000000,118813.000000,117183.000000
171139,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Trace Grade,Mt,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
171140,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Oceans Grade,Mt,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


### Country Traduction between IAMC country and Wiliam


In [89]:
# Open the text file containing the dictionary
with open('Create_Variable_Dict/country_dict.txt', 'r') as f:
    # Read the contents of the file
    dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
country_Wiliam_dict = ast.literal_eval(dict_str)

# Replace the region name by the ones adapted for Wiliam. 
scenario_variable_df['Region'] = scenario_variable_df['Region'].replace(country_Wiliam_dict)
scenario_variable_df

,Model,Scenario,Region,Variable,Unit,2005,2006,2007,2008,2009,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
0,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Agriculture Products,EJ/yr,0.028242,1.057450,1.265620,1.713050,1.934410,...,2.948760,2.824390,2.727550,2.623520,2.632230,2.47606,2.401450,2.344890,2.263520,2.172020
1,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Coal,EJ/yr,139.888000,142.536000,145.529000,148.652000,149.764000,...,239.392000,238.914000,242.655000,247.283000,252.159000,248.07700,251.761000,254.646000,253.793000,249.484000
2,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Oil,EJ/yr,202.483000,200.246000,200.156000,199.572000,199.148000,...,170.113000,162.348000,159.496000,158.317000,158.668000,151.41400,150.100000,147.932000,144.727000,139.896000
3,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Forestry Products,EJ/yr,37.127500,37.334300,37.479100,37.766300,37.924300,...,98.841200,98.722100,101.700000,105.143000,108.929000,106.94800,109.672000,112.871000,112.351000,111.132000
4,WILIAM,Baseline no policies no CO2tax,World,Primary Energy|Geothermal,EJ/yr,0.240555,0.249827,0.257823,0.267857,0.278548,...,7.077130,6.729580,6.799890,6.949150,7.092140,6.82465,6.960060,6.905350,6.840980,6.567220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171137,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Low Grade,Mt,128992.000000,127935.000000,126830.000000,125688.000000,124519.000000,...,66756.000000,65861.200000,64974.700000,64096.800000,63227.400000,62366.50000,61514.200000,60670.500000,59835.300000,59008.800000
171138,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Ultralow Grade,Mt,176000.000000,175990.000000,175978.000000,175963.000000,175945.000000,...,132301.000000,130580.000000,128867.000000,127163.000000,125469.000000,123787.00000,122116.000000,120458.000000,118813.000000,117183.000000
171139,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Trace Grade,Mt,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
171140,WILIAM,Baseline no policies no CO2tax,World,Resource|Iron|Oceans Grade,Mt,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


### Correct Naming Emissions Intensity, Primary Energy, Secondary Energy, Capacity


,Model,Scenario,Region,Variable,Unit,2005,2006,2007,2008,2009,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
116704,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Electricity|Gas Fuels|w/o CCS,TW/yr,0.0,0.004197,0.003319,0.002048,0.002519,...,1.192860e-03,7.710520e-04,1.060210e-03,1.277260e-03,1.443560e-03,5.030110e-04,1.133450e-03,1.738440e-03,7.749580e-04,3.536120e-04
116705,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Electricity|Gas Fuels|w/ CCS,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
116706,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Electricity|Geothermal|Deac...,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
116707,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Electricity|Liquid Fuels|w/...,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,1.516350e-08,7.704490e-09,1.015160e-08,1.320780e-08,1.016220e-08,3.428050e-09,6.201830e-09,8.115710e-09,3.613300e-09,1.627500e-09
116708,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Electricity|Liquid Fuels|w/...,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116991,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Solid Fossil|Wind|Offshore,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
116992,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Solid Fossil|Wind|Onshore,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
116995,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Solid Fossil|Hydrogen,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
116996,WILIAM,Baseline no policies no CO2tax,EU27,Capacity Additions|Solid Fossil|Solids|Biomass,TW/yr,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [114]:
def process_dataframe(df, column_name):
    """
    Process the DataFrame to replace 'TW' with 'GW' in the specified column (if it contains 'TW')
    and multiply all other columns by 1000 for the rows where the specified column contains 'TW'.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to check and replace 'TW' with 'GW'.

    Returns:
    pd.DataFrame: The processed DataFrame.
    """
    # Create a mask to identify rows where the specified column contains 'TW'
    mask = df[column_name].str.contains("TW")

    # Replace 'TW' with 'GW' in the specified column
    df.loc[mask, column_name] = df.loc[mask, column_name].str.replace("TW", "GW")

    # Multiply all other columns by 1000 for the rows where the mask is True
    cols_to_multiply = df.columns.difference(
        [column_name, "Model", "Scenario", "Region"]
    )
    df.loc[mask, cols_to_multiply] = df.loc[mask, cols_to_multiply] * 10**3

    return df


capacity_additions_rows = scenario_variable_df[
    scenario_variable_df["Variable"].str.contains("Capacity Additions")
]
capacity_additions_rows = process_dataframe(capacity_additions_rows, "Unit")
indexes = capacity_additions_rows.index
for index in indexes: 
    scenario_variable_df.loc[index] = capacity_additions_rows.loc[index]



Model                                    WILIAM
Scenario         Baseline no policies no CO2tax
Region                                    INDIA
Variable    Intensity|Emissions|Electricity|CO2
Unit                                      Mt/EJ
                           ...                 
2066                                   0.235328
2067                                   0.233475
2068                                   0.231808
2069                                   0.228856
2070                                    0.22533
Name: 151129, Length: 71, dtype: object

In [13]:
# Select rows with the name 'Emission'
emission_rows = scenario_variable_df[scenario_variable_df['Variable'].str.contains('Emission')]
emission_name_old = emission_rows['Variable'].to_list()
# List of Greenhouses Gases used in Wiliam 
# We differentiate the treatment of HFC and others GHGs because they are handle differently in IAMC Format. 
GHGs_to_check = ['CH4', 'N2O', 'PFC', 'SF6','CO2']
HFC_list = ['HFC134a', 'HFC23', 'HFC32', 'HFC125', 'HFC143a', 'HFC152a', 'HFC227ea', 'HFC245ca', 'HFC43-10']


# Iterate over the rows to create the name and the unit of emissions correctly. 
# The new list of variables and unit is then updated to the real dataframe.
variable_list = []
unit_list = []

for row_index, row in emission_rows[['Variable','Unit']].iterrows():
    specific_word = 'Emissions'
    ghg_bool = False
    for string in GHGs_to_check:
        
        if string in row['Variable']:
            
            index_emission = row['Variable'].find('Emissions') + len(specific_word)
            index_gas = row['Variable'].find(string)
            row['Variable'] = row['Variable'][:index_emission] + '|' + string + row['Variable'][index_emission:index_gas-1] + row['Variable'][index_gas+len(string):]            
            ghg = string
            ghg_bool = True
        
            break
    
    for string in HFC_list:
        if string in row['Variable']:
            
            index_emission = row['Variable'].find('Emissions') + len(specific_word)
            index_gas = row['Variable'].find(string)
            row['Variable'] = row['Variable'][:index_emission] + '|HFC|' + string + row['Variable'][index_emission:index_gas-1] + row['Variable'][index_gas+len(string):]
            
            break
    index_year = row['Unit'].find('/yr')
    if row['Unit'] == 'Mt/yr' or row['Unit'] == 'Gt/yr':
        if ghg_bool: 
            row['Unit'] = row['Unit'][:index_year] + ' ' + ghg + row['Unit'][index_year:]
        else : 
            row['Unit'] = row['Unit'][:index_year] + ' ' + string + row['Unit'][index_year:]
    variable_list.append(row['Variable'])
    unit_list.append(row['Unit'])

indexes = emission_rows.index

# Assign new values to specific rows in the column for the Variable and Unit 
scenario_variable_df.loc[indexes, 'Variable'] = variable_list
scenario_variable_df.loc[indexes, 'Unit'] = unit_list

In [14]:
# Change Primary Energy Naming
# Select the rows where the primary energy word 
energy_rows = scenario_variable_df[scenario_variable_df['Variable'].str.contains('Per Capita')]


energy_name_old = energy_rows['Variable'].to_list()

# Function to move the matched string after the specific word
def move_per_capita(s):
    specific_word = 'Per Capita'
    
    if specific_word in s:
        index_emission = s.find(specific_word) 
        
        s = s[:index_emission-1] + s[index_emission+len(specific_word):] + '|' + specific_word
   
    return s

# Apply the function to the 'Variable' column
energy_rows['Variable'] = energy_rows['Variable'].apply(move_per_capita)
energy_name_new = energy_rows['Variable'].to_list()

energy_per_capita_dict = dict(zip(energy_name_old, energy_name_new))

scenario_variable_df['Variable']= scenario_variable_df['Variable'].replace(energy_per_capita_dict)


C:\Users\AdGeeds\AppData\Local\Temp\ipykernel_6040\1791360945.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_rows['Variable'] = energy_rows['Variable'].apply(move_per_capita)


In [15]:
# Change Secondary Energy Naming 
secondary_energy_rows = scenario_variable_df[scenario_variable_df['Variable'].str.contains('Secondary Energy')]

secondary_energy_rows['Variable'].iloc[0]

'Secondary Energy|Electricity And Heat|Gas Fuels|w/o CCS'

In [16]:
# Change Secondary Energy Naming 
# Select the rows for Primary Energy Price 
price_energy_rows = scenario_variable_df[scenario_variable_df['Variable'].str.contains('Price')]
price_energy_rows = price_energy_rows[price_energy_rows['Variable'].str.contains('Primary Energy')]
price_energy_rows



,Model,Scenario,Region,Variable,Unit,2005,2006,2007,2008,2009,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
129,WILIAM,Baseline no policies no CO2tax,World,Price|Primary Energy|Electricity And Heat|Gas ...,Mdollars/EJ,6598.08,6740.28,6512.76,10124.6,5650.08,...,14764.5,12554.4,11570.2,14119.9,12782.2,12625.7,12742.4,13124.8,13316.6,11925.6
130,WILIAM,Baseline no policies no CO2tax,World,Price|Primary Energy|Electricity And Heat|Gas ...,Mdollars/EJ,6598.08,6740.28,6512.76,10124.6,5650.08,...,14764.5,12554.4,11570.2,14119.9,12782.2,12625.7,12742.4,13124.8,13316.6,11925.6
131,WILIAM,Baseline no policies no CO2tax,World,Price|Primary Energy|Electricity And Heat|Geot...,Mdollars/EJ,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,WILIAM,Baseline no policies no CO2tax,World,Price|Primary Energy|Electricity And Heat|Liqu...,Mdollars/EJ,9438.83,11007.00,12055.20,16609.6,10323.70,...,98919.9,103024.0,101209.0,110478.0,152725.0,150526.0,208087.0,287660.0,290324.0,271613.0
133,WILIAM,Baseline no policies no CO2tax,World,Price|Primary Energy|Electricity And Heat|Liqu...,Mdollars/EJ,9438.83,11007.00,12055.20,16609.6,10323.70,...,98919.9,103024.0,101209.0,110478.0,152725.0,150526.0,208087.0,287660.0,290324.0,271613.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,WILIAM,Baseline no policies no CO2tax,Rest of the World,Price|Primary Energy|Gas Fuels,Mdollars/EJ,0.00,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545,WILIAM,Baseline no policies no CO2tax,Rest of the World,Price|Primary Energy|Liquid Fuels,Mdollars/EJ,0.00,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546,WILIAM,Baseline no policies no CO2tax,Rest of the World,Price|Primary Energy|Hydrogen,Mdollars/EJ,0.00,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,WILIAM,Baseline no policies no CO2tax,Rest of the World,Price|Primary Energy|Solids|Biomass,Mdollars/EJ,0.00,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def remove_characters_between_indices(input_string, start_index, end_index):
    """
    Remove characters between two given indices of a string.

    Args:
    - input_string (str): The original string.
    - start_index (int): The index of the first character to remove (inclusive).
    - end_index (int): The index of the last character to remove (inclusive).

    Returns:
    - str: The modified string with characters removed between the specified indices.
    """
    if start_index >= end_index:
        return input_string  # Return the original string if start index is greater than or equal to end index
    
    # Ensure indices are within the range of the string
    start_index = max(start_index, 0)
    end_index = min(end_index, len(input_string) - 1)
    
    # Construct the new string by concatenating substrings before and after the characters to be removed
    return input_string[:start_index] + input_string[end_index + 1:]

def remove_string_between_pipe(input_string):
    """
    Remove the string between the second and third '|' characters in the input string.

    Args:
    - input_string (str): The original string.

    Returns:
    - str: The modified string with the content between '|' characters removed.
    """
    second_occurrence = input_string.find('|', input_string.find('|') + 1)
    third_occurrence = input_string.find('|', second_occurrence + 1)-1
    
    # Remove characters between the '|' characters using the previous function
    result = remove_characters_between_indices(input_string, second_occurrence, third_occurrence)

    # Remove CCS extension because it is not relevant 
    result = result.replace('|w/ CCS', '').replace('|w/o CCS', '')
    
    return result

price_energy_rows['Variable'] = price_energy_rows['Variable'].apply(remove_string_between_pipe)
indexes = price_energy_rows.index

# Assign new values to specific rows in the column for the Variable and Unit 
scenario_variable_df.loc[indexes, 'Variable'] = price_energy_rows['Variable']

### Check how to explain duplicates rows


In [18]:
# Check for duplicate rows
duplicate_rows = scenario_variable_df[scenario_variable_df.duplicated()]

# Display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

# Count the number of duplicate rows
num_duplicate_rows = duplicate_rows.shape[0]
print("Number of Duplicate Rows:", num_duplicate_rows)


Duplicate Rows:
         Model                        Scenario             Region  \
130     WILIAM  Baseline no policies no CO2tax              World   
133     WILIAM  Baseline no policies no CO2tax              World   
135     WILIAM  Baseline no policies no CO2tax              World   
138     WILIAM  Baseline no policies no CO2tax              World   
139     WILIAM  Baseline no policies no CO2tax              World   
...        ...                             ...                ...   
150210  WILIAM  Baseline no policies no CO2tax  Rest of the World   
150211  WILIAM  Baseline no policies no CO2tax  Rest of the World   
150212  WILIAM  Baseline no policies no CO2tax  Rest of the World   
150213  WILIAM  Baseline no policies no CO2tax  Rest of the World   
150214  WILIAM  Baseline no policies no CO2tax  Rest of the World   

                                                 Variable         Unit  \
130                        Price|Primary Energy|Gas Fuels  Mdollars/EJ   
133    

### Writing File


In [ ]:
# Write the new excel file in the File_Converted folder
folder_file_converted=os.path.join(folder_path,folder_name_converted)

# Get the correct naming for the converted file 
filename_with_extension = os.path.basename(last_file)
filename = splited_filename_with_extension[0] + 'converted' + splited_filename_with_extension[1]

# Remove duplicate rows
# scenario_variable_df.drop_duplicates(subset=['Region', 'Variable'],inplace=True)
scenario_variable_df.drop_duplicates(subset=['Region','Variable','Unit'],inplace=True)

# Write the following dataframe to excel
if splited_filename_with_extension[1] == '.xls' or splited_filename_with_extension[1] == '.xlsx':
    scenario_variable_df.to_excel(os.path.join(folder_file_converted,filename))
elif  splited_filename_with_extension[1] == '.csv': 
    scenario_variable_df.to_csv(os.path.join(folder_file_converted,filename),index=False)

print("Conversion Done")
print(time.time()-time0, "Seconds to run the code.")